In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('1957_2016.csv')

In [3]:
df = df.iloc[:, 1:]
df.head()

,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
0,10006,19570329,86308.500,0.018405,62.250,1412,1.117907,1.249717,0.134574,10.296745,...,0.013856,1.557681e-07,0.029167,0.013546,0.914592,0.802461,1.066005e-07,37.0,1.269299,0.371964
1,10014,19570329,3903.375,0.000000,2.625,1487,0.331304,0.109762,0.006667,7.032404,...,0.031389,8.383815e-06,0.071429,0.031165,0.943409,0.716406,1.105263e+00,39.0,1.358414,0.509329
2,10022,19570329,9841.000,-0.003846,6.375,1514,0.942052,0.887461,0.058761,7.294038,...,0.010066,5.360386e-06,0.020000,0.016886,0.993558,0.534808,2.387797e-07,NaN,NaN,NaN
3,10030,19570329,51901.000,0.077907,45.750,1207,0.886532,0.785940,-0.032065,9.516942,...,0.009523,1.090908e-07,0.017857,0.009104,0.736000,0.712647,9.202902e-08,37.0,NaN,NaN
4,10057,19570329,35250.000,-0.017730,69.250,500,1.229520,1.511719,-0.102811,8.421013,...,0.007661,2.386914e-07,0.007194,0.006748,0.889503,0.775954,6.631579e+00,35.0,1.125553,0.762989


In [4]:
df.shape

(3760033, 101)

In [4]:
# OLS-3 size book-to-market momentum
covariates = ['permno', 'DATE','RET', 'mvel1', 'bm', 'mom12m', 'mom1m', 'mom36m', 'mom6m']
ols3 = df[covariates]
 

In [5]:
ols3.dropna(inplace=True)
ols3.shape

<ipython-input-5-80b1c4c1bbd8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ols3.dropna(inplace=True)


(3219631, 9)

In [7]:
def r2_oos(y_true, y_pred, sample_weight=None):
    if sample_weight is not None:
        weight = sample_weight[:, np.newaxis]
    else:
        weight = 1.
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
                                                      dtype=np.float64)
    denominator = (weight * (y_true) ** 2).sum(axis=0,
                                                    dtype=np.float64)
   
    
    output_scores = 1 - (numerator / denominator)

    return output_scores

def recur_eval_dataset(dataset, year_start, year_stop):
    n = year_start + 12
    while n <= year_stop:
        train = dataset[dataset['DATE'] < year_start * 10000 + 101]
        vali = dataset[(dataset['DATE'] < n * 10000 + 101) & (dataset['DATE'] > year_start * 10000 + 101)]
        test = dataset[(dataset['DATE'] < (n + 1) * 10000 + 101) & (dataset['DATE'] > n * 10000 + 101)]
        yield train, vali, test
        year_start += 1
        n = year_start + 12


In [8]:
from sklearn.linear_model import LinearRegression
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from numpy import mean
# for ols3
score = []
for train, vali, test in recur_eval_dataset(ols3, 1975, 2016):
    x = train.iloc[:, 3:8]
    y = train.iloc[:, 2]
    model = LinearRegression()
    model.fit(x, y)
    val_x = vali.iloc[:, 3:8]
    val_y = vali.iloc[:, 2]
    predict_val = model.predict(val_x)
    #print(predict_val)
    r2_oos(val_y.values, predict_val)
    test_x = test.iloc[:, 3:8]
    test_y = test.iloc[:, 2]
    predict_test = model.predict(test_x)
    score.append(r2_oos(test_y.values, predict_test))
mean(score)


0.0013682798938961219

In [9]:
ols = df.iloc[:, 1:]
ols.dropna(inplace=True)
ols.head()

,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,idiovol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
4,19570329,35250.00,-0.017730,69.25,500,1.229520,1.511719,-0.102811,8.421013,0.025453,...,0.007661,2.386914e-07,0.007194,0.006748,0.889503,0.775954,6.631579e+00,35.0,1.125553,0.762989
23,19570329,283036.00,0.057692,55.00,5443,1.806498,3.263435,0.006612,10.459776,0.019643,...,0.016233,7.592996e-08,0.022167,0.012118,0.421592,0.226673,2.443123e-07,33.0,0.650905,-0.584311
25,19570329,2670133.50,0.014799,177.75,62893,0.389741,0.151898,0.019737,11.948152,0.006526,...,0.005598,2.747520e-09,0.009972,0.004336,0.320722,0.034650,1.202819e-06,48.0,0.544756,-0.238968
28,19570329,17994.25,0.134132,46.75,431,0.677784,0.459391,-0.113021,8.688643,0.030575,...,0.007412,4.835197e-07,0.009036,0.010821,0.779174,0.746046,1.105263e+00,38.0,0.805466,0.225861
30,19570329,48112.50,0.020000,38.25,1283,0.948980,0.900563,-0.194429,9.489297,0.025403,...,0.012026,2.513525e-07,0.019231,0.011344,0.694251,0.744191,1.200925e-07,33.0,0.925097,0.138460


In [10]:
# for ols
score = []
for train, vali, test in recur_eval_dataset(ols, 1975, 2016):
    x = train.iloc[:, 1:].drop(columns='RET')
    y = train.iloc[:, 2]
    model = LinearRegression()
    model.fit(x, y)
    val_x = vali.iloc[:, 1:].drop(columns='RET')
    val_y = vali.iloc[:, 2]
    predict_val = model.predict(val_x)
    print('---------')
    r2_oos(val_y.values, predict_val)
    test_x = test.iloc[:, 1:].drop(columns='RET')
    test_y = test.iloc[:, 2]
    predict_test = model.predict(test_x)
    score.append(r2_oos(test_y.values, predict_test))
mean(score)

LinearRegression()

---------


-0.07266697164129399

LinearRegression()

---------


...

-0.029295141811257475

LinearRegression()

---------


-0.061685500947183725

-0.5225223405826438